# Automated GenAI Newsletter Generator

# Author: Ravindra Bharathi
# Email: ravindra.graicells@gmail.com
# Date: 2025-02-27

## Setup Environment
### Install dependencies:


In [1]:
!pip install arxiv python-dotenv PyGithub python-pptx pandas --quiet

In [2]:
!pip install google-generativeai --upgrade --quiet

## Configuration


In [3]:
from google.colab import userdata
import os
import ssl
import re

from github import Github
import arxiv

import time
import google.generativeai as genai
#from google.generativeai.types import RateLimitError #Import RateLimitError
from google.api_core.exceptions import ResourceExhausted




#Credentials
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
github_token = userdata.get('GITHUB_TOKEN')
email_password = userdata.get('EMAIL_PASSWORD')
email_id=userdata.get('EMAIL_ID')

# Initialize clients
github = Github(github_token)

In [4]:
# Configure Gemini API
genai.configure(api_key=GOOGLE_API_KEY)
generation_config = genai.types.GenerationConfig(
    temperature=0.5,
    top_p=1,
    top_k=1,
    max_output_tokens=2048,
)
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]




In [5]:
#check which models are available
print('Available models:')
models=genai.list_models()
for model in models:
  print(model.name)

Available models:
models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental
models/embedding-001
models/text-embedding-004
models/aq

In [6]:
#instantiate the model - gemini-2.0-flash
model = genai.GenerativeModel(model_name="models/gemini-2.0-flash",
                             generation_config=generation_config,
                            safety_settings=safety_settings)

## Core Functions
### 1. Content Aggregation
Fetch research papers and GitHub repos:

In [7]:
def fetch_ai_content():
    """Fetch latest GenAI research and projects"""

    # Get arXiv papers
    arxiv_client = arxiv.Client()
    search = arxiv.Search(
        query="generative AI",
        max_results=5,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    papers = [result for result in arxiv_client.results(search)]

    # Get GitHub repos
    repos = github.search_repositories(
        query="generative AI created:>2025-01-01",
        sort="updated"
    )[:3]

    return {
        "papers": papers,
        "repos": repos
    }

### 2. Analysis
Generate technical summaries:

In [8]:
def analyze_content(content):
    """Process content with GPT-4"""

    # Generate paper summaries
    paper_summaries = []
    for paper in content['papers']:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "system",
                "content": f"Summarize this AI paper in 3 bullet points:\n{paper.summary}"
            }],
            temperature=0.3
        )
        paper_summaries.append(response.choices[0].message.content)

    # Analyze GitHub repos
    repo_analyses = []
    for repo in content['repos']:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "system",
                "content": f"Explain the technical significance of {repo.name}:\n{repo.description}"
            }],
            temperature=0.5
        )
        repo_analyses.append(response.choices[0].message.content)

    return {
        "papers": paper_summaries,
        "repos": repo_analyses
    }

In [9]:
def analyze_content(content):
    """Process content with Gemini Pro"""

    # Generate paper summaries
    paper_summaries = []
    for paper in content['papers']:
        def generate_gemini_text_with_retry(prompt):
            retries = 3
            wait_time = 1
            for attempt in range(retries):
                try:
                    response = model.generate_content(prompt)
                    return response.text
                except ResourceExhausted as e:
                    if attempt == retries - 1:
                        raise e
                    else:
                        time.sleep(wait_time)
                        wait_time *= 2

        prompt = f"Summarize this AI paper in 3 bullet points:\n{paper.summary}"
        summary = generate_gemini_text_with_retry(prompt)
        paper_summaries.append(summary)


    # Analyze GitHub repos
    repo_analyses = []
    for repo in content['repos']:
        def generate_gemini_text_with_retry(prompt):
            retries = 3
            wait_time = 1
            for attempt in range(retries):
                try:
                    response = model.generate_content(prompt)
                    return response.text
                except ResourceExhausted as e:
                   if attempt == retries - 1:
                       raise e
                   else:
                       time.sleep(wait_time)
                       wait_time *= 2
        prompt = f"Explain the technical significance of {repo.name}:\n{repo.description}"
        analysis = generate_gemini_text_with_retry(prompt)
        repo_analyses.append(analysis)

    return {
        "papers": paper_summaries,
        "repos": repo_analyses
    }

### 3. Newsletter Generation
Create formatted email content:

In [26]:
def generate_newsletter(analysis):
    """Create HTML newsletter with Gemini Models"""

    # Generate header
    header = "# 🚀 Weekly GenAI Digest\n\n"

    # Top papers section
    papers_section = "## 📄 Top Research Papers\n"
    papers_section += "\n".join([f"- {summary}" for summary in analysis['papers']])

    # GitHub projects section
    repos_section = "\n\n## 💻 Trending Code Repos\n"
    repos_section += "\n".join([f"- {analysis}" for analysis in analysis['repos']])

    # Final assembly
    full_content = header + papers_section + repos_section


    # Format with Gemini
    def generate_gemini_text_with_retry(prompt):
        retries = 3
        wait_time = 1
        for attempt in range(retries):
            try:
                response = model.generate_content(prompt)
                return response.text
            except ResourceExhausted as e:
                if attempt == retries - 1:
                    raise e
                else:
                    time.sleep(wait_time)
                    wait_time *= 2

    prompt = f"""You are an expert in HTML and newsletter formatting. Your task is to convert the following plain text newsletter content into well-structured, readable HTML. The output should be a complete HTML document with inline CSS styles for basic formatting. Make sure the format is suitable for sending as an HTML email.

Here are the formatting guidelines:

*   **Headings:**
    *   Top-level headings (starting with `#`) should be wrapped in `<h1>` tags.
    *   Second-level headings (starting with `##`) should be wrapped in `<h2>` tags.

*   **Lists:**
    *   Bullet points (lines starting with `-`) should be formatted as unordered lists (`<ul>`). Each bullet point should be inside a list item (`<li>`) tag.
    *  If a bullet point contains a second bullet point (a line starting with `*`), the second list should be nested in the parent list item.

*   **Inline Styling:**
    *   Use inline CSS styles for basic formatting, such as `font-family`, `line-height`, `margin`, `color`, and `padding`.
*   **General Structure:**
    *   Ensure that the output is a valid HTML document that includes `<html>`, `<head>`, `<style>`, and `<body>` tags.

*   **Other content:**
    *   If there is other content that isn't a heading or list, wrap it in `<p>` tags.
* Make sure that the content can be sent as html email in gmail

Here is the newsletter content to format:\n{full_content}"""
    formatted_content = generate_gemini_text_with_retry(prompt)


    return formatted_content

## Execution Pipeline
Run the complete workflow:

In [ ]:
# Generate This Week's Issue
content = fetch_ai_content()
analysis = analyze_content(content)


In [27]:
newsletter = generate_newsletter(analysis)

print("Newsletter Generated!\n")
print(newsletter)


Newsletter Generated!

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weekly GenAI Digest</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 20px;
            color: #333;
        }
        h1 {
            font-size: 2em;
            margin-bottom: 0.5em;
            color: #0056b3;
        }
        h2 {
            font-size: 1.5em;
            margin-top: 1em;
            margin-bottom: 0.5em;
            color: #0056b3;
        }
        ul {
            margin-bottom: 1em;
            padding-left: 20px;
        }
        li {
            margin-bottom: 0.5em;
        }
        p {
            margin-bottom: 1em;
        }
    </style>
</head>
<body>
    <h1 style="font-size: 2em; margin-bottom: 0.5em; color: #0056b3;">🚀 Weekly GenAI Digest</h1>

    <h2 style="font-size: 1.5em; margin-top: 

In [28]:
newsletter=newsletter.strip('```html\n').strip('\n```')

In [29]:
newsletter

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Weekly GenAI Digest</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            line-height: 1.6;\n            margin: 20px;\n            color: #333;\n        }\n        h1 {\n            font-size: 2em;\n            margin-bottom: 0.5em;\n            color: #0056b3;\n        }\n        h2 {\n            font-size: 1.5em;\n            margin-top: 1em;\n            margin-bottom: 0.5em;\n            color: #0056b3;\n        }\n        ul {\n            margin-bottom: 1em;\n            padding-left: 20px;\n        }\n        li {\n            margin-bottom: 0.5em;\n        }\n        p {\n            margin-bottom: 1em;\n        }\n    </style>\n</head>\n<body>\n    <h1 style="font-size: 2em; margin-bottom: 0.5em; color: #0056b3;">🚀 Weekly GenAI Digest</h1>\n\n    <h2 style="font-size: 1.5em; mar

## Email Distribution
Send newsletter to subscribers:

In [30]:
import smtplib
import ssl
from email.mime.text import MIMEText

def send_newsletter(recipients):
    """Distribute via Gmail"""

    msg = MIMEText(newsletter, 'html')
    msg['Subject'] = 'Your Weekly GenAI Update'
    msg['From'] = 'sender@gmail.com'
    msg['To'] = ', '.join(recipients)
    msg.replace_header('Content-Type', 'text/html; charset="utf-8"')


    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(email_id, email_password)
        server.sendmail(
            msg['From'], recipients, msg.as_string()
        )

# Send Email
test_recipients = ["youremail@gmail.com"]  # Replace with correct email
send_newsletter(test_recipients)
print("Newsletter sent successfully!")

Newsletter sent successfully!
